### TP_5
### Objetivo
### Implementar el detector de fondo naive usando la mediana como estimador El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo
### Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento
### Comparar con alguno de los métodos vistos en la practica basados basados en mezcla de gaussianas

In [2]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
#%matplotlib

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt

import typing
import builtins as _mod_builtins
import math
from collections import deque

In [3]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [3]:
# Abrimos el archivo
#-------------------
#filename = 'vtest.avi'
#capture = cv.VideoCapture(filename)

#if not capture.isOpened:
 #   print('Falla al abrir el archivo: ' + filename)
  #  exit(0)


capture = cv.VideoCapture(0)
if capture.isOpened() == 0:
    exit(-1)

# Set the video resolution to HD720 (2560*720)
capture.set(cv.CAP_PROP_FRAME_WIDTH, 2560)
capture.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    N=10 # observe que a mayor N cuando me muevo, la dteeccion marca bordes del entorno que no tuvieron desplazamiento...
    
    fgMask = backSub.apply(frame,N)
     
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

In [4]:
# Abrimos el archivo
#-------------------
#filename = 'vtest.avi'
#capture = cv.VideoCapture(filename)

#if not capture.isOpened:
 #   print('Falla al abrir el archivo: ' + filename)
  #  exit(0)


capture = cv.VideoCapture(0)
if capture.isOpened() == 0:
    exit(-1)

# Set the video resolution to HD720 (2560*720)
capture.set(cv.CAP_PROP_FRAME_WIDTH, 2560)
capture.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
     
    # Aplicar el filtro de media
    mean = cv.blur(frame,(15,15))

    # Aplicar el filtro de mediana
    median = cv.medianBlur(frame,25)
    
    # Aplicamos un suavizado Gaussiano de 3x3 y de 5x5
    low_sigma = cv.GaussianBlur(frame, (3,3), 0, 0)
    high_sigma = cv.GaussianBlur(frame, (5,5), 0, 0)

    # Calculamos el DoG por sustracción
    dog = low_sigma - high_sigma

    # Aplico filtro de mediana
    median2 = cv.medianBlur(dog,5)
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    
    # Mostrar la imagen
    cv.imshow('Original',frame)
    cv.imshow('Media',mean)
    cv.imshow('Mediana',median)
    cv.imshow('Mediana',median2)
    

    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()